# Hyper parameter search for CQL

In [23]:
!pip install d3rlpy

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
# plt.style.use('matplotlibrc')

# from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [25]:
import numpy as np
import torch
import random
import pandas
from Python.data_sampler import *

In [26]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset, states.numpy(), actions.numpy(), rewards.numpy()

We can build the dataset from there, just like this, and split into train and test sets.

In [27]:
dataset, states, actions, rewards = get_dataset([i for  i in range(50)], path="collected_data/rl_stochpid.txt")

start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02 -3.69999953e-02
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.50429671e-01 -4.92727243e-01 -5.31666025e-03]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.25610892e-01 -3.35999953e-02
 -2.42000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08749986e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90489108e-01 -5.87999953e-02
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.76979602e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48010892e-01  4.64000047e-02
 -1.04000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.83151637e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.08389108e-01  3.32000047e-02
 -2.02000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.25137655

[ 0.00000000e+00  7.95731469e+08  3.84410892e-01 -2.65999953e-02
  1.95999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 40 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.37889108e-01  4.72000047e-02
 -6.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.04730803e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 41 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.74610892e-01  1.40000047e-02
  7.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.58596924e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 42 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.29289108e-01 -1.49999953e-02
  1.51999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 43 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -7.03891077e-02 -1.45999953e-02
  2.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.45547944e-

In [28]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -138.04016,
 'std': 107.28185,
 'min': -400.4336,
 'max': 0.0,
 'histogram': (array([ 2,  1,  2,  1,  0,  0,  0,  2,  4,  1,  0,  1,  5,  3,  3,  3,  5,
         15,  1,  1]),
  array([-400.4336  , -380.41193 , -360.39023 , -340.36856 , -320.34686 ,
         -300.3252  , -280.30353 , -260.28183 , -240.26016 , -220.23848 ,
         -200.2168  , -180.19511 , -160.17343 , -140.15176 , -120.13008 ,
         -100.1084  ,  -80.086716,  -60.06504 ,  -40.043358,  -20.021679,
            0.      ], dtype=float32))}

In [29]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [30]:
from d3rlpy.algos import CQL
from d3rlpy.models.encoders import VectorEncoderFactory

from d3rlpy.preprocessing import MinMaxActionScaler
import random
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)

from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer
import pickle as pkl
from statistics import harmonic_mean as hm

## Perform a random search on hyper params. 

As good fqe on both stochastic and deterministic data is needed for good performance during the evaluation step in the simulator, we take the harmonic mean of both fqes to understand the performance on each dataset. The harmonic mean is large only when each number whose mean is being taken is large. Therefore, a large harmonic mean corresponds to large numbers whose mean is being taken.


In [33]:

# The number of times we would wanna do a random search for hyper-params. In 
# every random search, we randomly pick a new set of hyper-params.
num_search_iterations = 20
largest_fqe = -np.inf

for i in range(num_search_iterations):

    random.seed()

    actor_lr_this_iter = random.uniform(1e-5, 1e-2)
    critic_lr_this_iter = random.uniform(1e-5, 1e-2)
    temp_lr_this_iter = random.uniform(1e-5, 1e-4)
    n_steps_this_iter = random.choice([1, 3, 5, 7])

    actor_encoder = VectorEncoderFactory(hidden_units=[12, 24, 36, 24, 12],
                                          activation='relu', use_batch_norm=True, dropout_rate=0.2)
    critic_encoder = VectorEncoderFactory(hidden_units=[12, 24, 24, 12],
                                          activation='relu', use_batch_norm=True, dropout_rate=0.2)

    print("search iteration: ", i)
    print("using hyper params: ", [actor_lr_this_iter, critic_lr_this_iter, 
                                   temp_lr_this_iter, n_steps_this_iter])

    model = CQL(q_func_factory='qr', #qr -> quantile regression q function, but you don't have to use this
                reward_scaler='standard',
                actor_encoder_factory = actor_encoder,
                critic_encoder_factory = critic_encoder,
                action_scaler=action_scaler,
                actor_learning_rate=actor_lr_this_iter, 
                critic_learning_rate=critic_lr_this_iter,
                temp_learning_rate=temp_lr_this_iter,
                n_steps=n_steps_this_iter, 
                use_gpu=True) #change it to true if you have one
    model.build_with_dataset(dataset)

    model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=50, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })
    
    ope_dataset, states_ope, actions_ope, rewards_ope = get_dataset([i+100 for i  in range(10)], 
                                                                    path="collected_data/rl_stochpid.txt") #change if you'd prefer different chunks
    ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

    fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
    history_stoch = fqe.fit(ope_train_episodes,
        eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })
    
    ope_dataset, states_ope, actions_ope, rewards_ope = get_dataset([i+200 for i in range(10)], 
                                                                    path="collected_data/rl_stochpid.txt") #change if you'd prefer different chunks
    ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

    fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
    history_det = fqe.fit(ope_train_episodes,
        eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })
    
    if(hm([history_stoch[-1][1]["ave_value"] + 50, 
           history_det[-1][1]["ave_value"] + 50]) \
                                                          > largest_fqe):
        largest_fqe = hm([history_stoch[-1][1]["ave_value"] + 50, 
                         history_det[-1][1]["ave_value"] + 50])

        # Save the hyper-params
        hyperparams = [actor_lr_this_iter, critic_lr_this_iter, 
                       temp_lr_this_iter, n_steps_this_iter]

        with open("hyperparams_cql.pkl", "wb") as f:
            print("most optimal hyper params for cql at this point: ", hyperparams)
            pkl.dump(hyperparams, f)

        # Save model and policy
        model.save_model("model_hyperparams_cql.pt")
        model.save_policy("policy_hyperparams_cql.pt")


search iteration:  0
using hyper params:  [0.00469557651656814, 0.003304999882985713, 4.52538497359818e-05, 7]
2022-04-17 14:51.01 [debug    ] RoundIterator is selected.
2022-04-17 14:51.01 [info     ] Directory is created at d3rlpy_logs/CQL_20220417145101
2022-04-17 14:51.01 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-17 14:51.01 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-17 14:51.01 [warning  ] Skip building models since they're already built.
2022-04-17 14:51.01 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220417145101/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'vector', 'params': {'hidden_units': [12, 24, 36, 24, 12], 'activation': 'relu', 'use_batch_norm': True, 'dropout_rate': 0.2, 'use_dense': False}}, 'actor_learning_rate': 0.00469557651656814, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 

Epoch 1/6:   0%|          | 0/169 [00:00<?, ?it/s]

2022-04-17 14:51.07 [info     ] CQL_20220417145101: epoch=1 step=169 epoch=1 metrics={'time_sample_batch': 0.00036236238197462096, 'time_algorithm_update': 0.029904378236398188, 'temp_loss': 4.738555341077274, 'temp': 0.9959946380564447, 'alpha_loss': -22.22049955785627, 'alpha': 1.0091362232287255, 'critic_loss': 157.25083864652194, 'actor_loss': -0.46273912919255406, 'time_step': 0.030343302608241696, 'td_error': 9.133720136999088, 'init_value': -4.455194473266602, 'ave_value': -3.4397618988397958} step=169
2022-04-17 14:51.07 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417145101/model_169.pt


Epoch 2/6:   0%|          | 0/169 [00:00<?, ?it/s]

2022-04-17 14:51.12 [info     ] CQL_20220417145101: epoch=2 step=338 epoch=2 metrics={'time_sample_batch': 0.00035676025074614576, 'time_algorithm_update': 0.031510051185562764, 'temp_loss': 4.935345353459465, 'temp': 0.9882349967956543, 'alpha_loss': -19.686638499152732, 'alpha': 1.027395603219433, 'critic_loss': 90.78072853765543, 'actor_loss': 4.483377108207116, 'time_step': 0.03194766213907998, 'td_error': 1.2886037594896624, 'init_value': -9.449234008789062, 'ave_value': -8.013484366197844} step=338
2022-04-17 14:51.12 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417145101/model_338.pt


...

2022-04-17 15:04.34 [info     ] FQE_20220417150430: epoch=9 step=693 epoch=9 metrics={'time_sample_batch': 0.00017098018101283481, 'time_algorithm_update': 0.004550868814641779, 'loss': 0.0021742461566728625, 'time_step': 0.004793848310198102, 'init_value': -0.17744670808315277, 'ave_value': -0.16381982802853895, 'soft_opc': nan} step=693


2022-04-17 15:04.34 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417150430/model_693.pt


Epoch 10/10:   0%|          | 0/77 [00:00<?, ?it/s]

2022-04-17 15:04.34 [info     ] FQE_20220417150430: epoch=10 step=770 epoch=10 metrics={'time_sample_batch': 0.00017018751664595172, 'time_algorithm_update': 0.004591502152480088, 'loss': 0.0023497399789365854, 'time_step': 0.004828338499193068, 'init_value': -0.1851753443479538, 'ave_value': -0.18074607690313815, 'soft_opc': nan} step=770


2022-04-17 15:04.34 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417150430/model_770.pt


## Reading hyper params from file

In [34]:
with open("hyperparams_cql.pkl", "rb") as f:
    data = pkl.load(f)

print(data)

[0.00469557651656814, 0.003304999882985713, 4.52538497359818e-05, 7]
